<a href="https://colab.research.google.com/github/Debottam/MachinLearningEx/blob/master/TripletVAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import tensorflow as tf
from tensorflow import keras

print("tf version: ", tf.__version__)
print("keras version: ", keras.__version__)

tf version:  2.3.0
keras version:  2.4.0


In [5]:
#triplet

class Tripletnet(keras.Model):
    def __init__(self, embeddingnet):
        super().__init__(**kwargs)
        self.embeddingnet = embeddingnet
    def call(self, x, y, z):
        latent_x,mean_x,logvar_x = self.embeddingnet(x)
        latent_y,mean_y,logvar_y = self.embeddingnet(y)
        latent_z,mean_z,logvar_z = self.embeddingnet(z)
        dist_a = tf.square(mean_x - mean_y)
        dist_b = tf.square(mean_x - mean_z)
        return latent_x,mean_x,logvar_x,\
            latent_y,mean_y,logvar_y,\
            latent_z,mean_z,logvar_z,\
            dist_a, dist_b

In [6]:
# kl_loss
def kl_reconstruction_loss(x, recon_x, mu, log_var):
    #reco_loss = tf.reduce_mean(tf.square(x - recon_x))
    kl_loss = 1 + log_var - tf.square(mu) - tf.exp(log_var)
    kl_loss = tf.reduce_sum(kl_loss, axis= -1)
    kl_loss *= -0.5
    return tf.reduce_mean(kl_loss)

# triplet loss
def triplet_loss(anchor, positive, negative):
    positive_dist = tf.reduce_mean(tf.square(anchor - positive), axis=1)
    negative_dist = tf.reduce_mean(tf.square(anchor - negative), axis=1)
    return tf.maximum(positive_dist - negative_dist + alpha, 0.)

In [7]:
input_dim = 80
encoding_dim = 10
inputs = keras.Input((None, input_dim))

class _Encoder(keras.Model):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.layer_1 = keras.layers.Dense(256, activation="relu")
        self.layer_2 = keras.layers.Dense(128, activation="relu")
        self.layer_3 = keras.layers.Dense(64, activation="relu")

    def call(self,inputs):
        x = self.layer_1(inputs)
        x = self.layer_2(x)
        x = self.layer_3(x)
        self.mean = keras.layers.Dense(encoding_dim, name="z_mean")(x)
        self.log_var = keras.layers.Dense(encoding_dim, name="z_log_var")(x)
        self.z = keras.layers.Lambda(sampling, output_shape=(encoding_dim,), name='z')([self.mean, self.log_var])
        return self.z, self.mean, self.log_var

class _Decoder(keras.Model):
    def __init__(self):
        super(self).__init__()
        self.layer_1 = keras.layers.Dense(128, activation="relu")
        self.layer_2 = keras.layers.Dense(256, activation="relu")
        self.layer_3 = keras.layers.Dense(80, activation="sigmoid")
        
    def call(self,inputs):
        x = self.layer_1(inputs)
        x = self.layer_2(x)
        output = self.layer_3(x)
        return output

In [13]:
_Encoder().build()

TypeError: ignored

In [18]:
# triplet_VAE
custMetric = keras.metrics.Mean()
class tripletVAE (keras.Model):
  def __init__(self, **kwargs):
    super().__init__(**kwargs)
    self._Encoder = _Encoder()
    self._Decoder = _Decoder()
  
  def call(self,inputs):
    x, y , z = inputs
    latent_x,mean_x,logvar_x = self._Encoder(x)
    latent_y,mean_y,logvar_y = self._Encoder(y)
    latent_z,mean_z,logvar_z = self._Encoder(z)
    dist_a = tf.square(mean_x - mean_y)
    dist_b = tf.square(mean_x - mean_z)
    recon_x = self._Decoder(latent_x)
    recon_y = self._Decoder(latent_y)
    kl_loss_x = kl_reconstruction_loss(x, recon_x, mean_x, logvar_x)
    kl_loss_y = kl_reconstruction_loss(y, recon_y, mean_y, logvar_y)
    triplet_loss = triplet_loss(mean_x,mean_y,mean_z)
    loss = kl_loss_x + kl_loss_y + triplet_loss
    self.add_loss (loss)
    self.add_metric (custMetric(triplet_loss, name='triplet'))
    return recon_x, recon_y
  
